In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import randint

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV , RandomizedSearchCV , cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error, mean_squared_error, r2_score , root_mean_squared_error
from sklearn.model_selection import cross_val_score, StratifiedKFold


import joblib
import os

# Import Functions

In [4]:
def load_data():
    '''
    A function for loading csv data into dataframe df.
    '''

    #Location of csv file
    csv_file = '../airpollutionlevels/raw_data/air_pollution_data_upd.csv'

    #Loading csv file into df dataframe
    df = pd.read_csv(csv_file)

    return df

def clean_data(df):
    '''
    A function to clean raw data:
    - Dropping unuseful columns
    - Dropping rows with year = NA
    - Dropping rows where pm10_concentration AND pm25_concentration AND no2_concentration are NA
    '''

    #Dropping columns: web_link, reference, iso3, who_ms, population_source, version, pm10_tempcov, pm25_tempcov, no2_tempcov
    df.drop(columns=['web_link',
                     'reference',
                     'iso3',
                     'who_ms',
                     'population_source',
                     'version',
                     'pm10_tempcov',
                     'pm25_tempcov',
                     'no2_tempcov'],
            inplace=True)

    #Dropping rows where year is NA (3 rows for India)
    df.dropna(subset=['year'], inplace=True)

    #Dropping rows where pm10_concentration AND pm25_concentration AND no2_concentration are NA
    df.dropna(how='all', subset=['pm10_concentration', 'pm25_concentration', 'no2_concentration'], inplace=True)

    return df

In [9]:
def simplify_stations(station_type):
    '''
    Simplifies the station type string by removing duplicates and sorting.

    Args:
    - station_type (str): A string containing station types separated by ', ' e.g. Urban, urban, urban.

    Returns:
    - str: Simplified station types joined into a single string e.g "Urban, urban, urban" returns "Urban"

    If station_type is NaN (missing), returns 'unknown'.'''

    if pd.isna(station_type):
        return "unknown"
    unique_types = sorted(set(station_type.split(', ')))
    return ', '.join(unique_types)

def simplified_station_type(df):
    '''
    Adds a new column 'simplified_station_type' to the DataFrame 'df' based on simplifying 'type_of_stations'.

    Args:
    - df (pandas.DataFrame): The DataFrame containing the column 'type_of_stations' to be simplified.

    Returns:
    - pandas.DataFrame: The input DataFrame 'df' with an additional column 'simplified_station_type'.

    This function applies the 'simplify_stations' function to each value in the 'type_of_stations' column
    and stores the simplified result in a new column 'simplified_station_type'
    '''

    df['type_of_stations'] = df['type_of_stations'].astype('string') #converts type_of_stations column into a string in order to apply simplify_stations function
    df['simplified_station_type'] = df['type_of_stations'].apply(simplify_stations)
    return df

def impute_stations(df):
    '''
    Imputes the values of missing type_of_stations based on similar pollution metrics of know types of stations using KNN imputer'''

    #first simplify station names using simplified_station_type function
    simplified_station_type(df)

    # Manually map known types of stations to numerical labels from stations3 df
    type_mapping = {
        'Unknown': np.nan, #will need this to be nan for imputer to work
        'Urban': 1,
        'Rural': 2,
        'Suburban': 3,
        'Suburban, Urban': 4,
        'Rural, Urban': 5,
        'Rural, Suburban, Urban': 6,
        'Rural, Suburban': 7,
        'Background': 8,
        'Residential And Commercial Area': 9,
        'Traffic': 10,
        'Residential And Commercial Area, Urban Traffic': 11,
        'Background, Traffic': 12,
        'Industrial': 13,
        'Residential And Commercial Area, Urban Traffic': 14,
        'Industrial, Urban': 15,
        'Industrial, Rural, Urban': 16,
        'Residential': 17,
        'Fond Urbain, Traffic': 18,
        'Residential - industrial': 19
    }

    df['encoded_station_type'] = df['simplified_station_type'].map(type_mapping) # encode simpified_station_type column to feed into KNN imputer

    # Select features for imputation
    features = ['population', 'pm25_concentration', 'encoded_station_type'] #features to be learned by imputer

    # Perform KNN imputation
    imputer = KNNImputer(n_neighbors=5) #instantiate imputer
    df_imputed = imputer.fit_transform(df[features]) #returns array with learned features

    # Assign imputed values back to DataFrame
    df['encoded_station_type_imputed'] = df_imputed[:, -1]  # Assuming encoded_station_type is the last column after imputation

    # Revert encoded_station_type back to original categorical values
    reverse_mapping = {v: k for k, v in type_mapping.items() if pd.notna(v)}  # Reverse mapping excluding NaNs. source >> https://stackoverflow.com/questions/483666/reverse-invert-a-dictionary-mapping

    df['final_station_type'] = df['encoded_station_type_imputed'].round().astype(int).map(reverse_mapping).fillna(np.nan)

    df.to_csv('../airpollutionlevels/raw_data/data_lib_rf.csv', index=False) #save the transformed data to a csv file to be used in the model for city prediction
    return df

In [11]:
def encode_scale_data_rf(df):
    # Drop rows with missing values in critical columns
    df = df.dropna(subset=['pm25_concentration', 'country_name', 'year', 'population', 'latitude', 'longitude'])

    # Convert 'year' to integer
    df = df.copy()  # Make a copy to avoid modifying the original DataFrame slice
    df['year'] = df['year'].astype(int)

    # Columns to drop if they exist in the DataFrame
    columns_to_drop = ['pm10_concentration', 'no2_concentration', 'type_of_stations',
                       'simplified_station_type', 'encoded_station_type', 'encoded_station_type_imputed']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1)

    # Drop 'city' column if it exists
    if 'city' in df.columns:
        df = df.drop(columns='city', axis=1)

    # Reset index to ensure it's sequential and clean
    df = df.reset_index(drop=True)

    # Add a unique identifier
    df['unique_id'] = range(len(df))

    # Define the columns for encoding and scaling
    categorical_cols = ['who_region', 'country_name', 'final_station_type']
    numeric_cols = ['population', 'latitude', 'longitude']

    # Instantiate encoders and scalers
    onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
    scaler = StandardScaler()
    pm25_scaler = StandardScaler()

    # Fit the pm25_concentration scaler separately and transform
    df['pm25_concentration'] = pm25_scaler.fit_transform(df[['pm25_concentration']])

    # Save the pm25_scaler for future inverse transformation
    pm25_scaler_filename = '../airpollutionlevels/models/pm25_scaler.pkl'
    joblib.dump(pm25_scaler, pm25_scaler_filename)

    # Pipeline for encoding and scaling
    preprocessor = ColumnTransformer(
        transformers=[
            ('onehot', onehot_encoder, categorical_cols),
            ('scaler', scaler, numeric_cols)
        ],
        remainder='passthrough'  # Keep the year and unique_id unchanged
    )

    # Apply transformations (excluding the already scaled 'pm25_concentration')
    transformed_data = preprocessor.fit_transform(df.drop(columns=['pm25_concentration']))

    # Get the feature names after one-hot encoding
    ohe_feature_names = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_cols)

    # Construct the final DataFrame columns
    final_columns = list(ohe_feature_names) + numeric_cols + ['year', 'unique_id']

    # Create the final DataFrame without the pm25_concentration column
    df_transformed = pd.DataFrame(transformed_data, columns=final_columns)

    # Add the scaled pm25_concentration back to the DataFrame
    df_transformed['pm25_concentration'] = df['pm25_concentration'].values

    # Save the transformed data to a CSV file
    df_transformed.to_csv('../airpollutionlevels/raw_data/air_pollution_data_encoded_rf.csv', index=False)

    return df_transformed

# Data import and Preproccess

In [8]:
data = load_data()
data = clean_data(data)
data = impute_stations(data)


In [12]:
data_enc = encode_scale_data_rf(data)

In [13]:
data_enc

,who_region_2_Amr,who_region_3_Sear,who_region_4_Eur,who_region_5_Emr,who_region_6_Wpr,who_region_7_NonMS,country_name_Albania,country_name_Algeria,country_name_Argentina,country_name_Australia,...,final_station_type_Suburban,"final_station_type_Suburban, Urban",final_station_type_Traffic,final_station_type_Urban,population,latitude,longitude,year,unique_id,pm25_concentration
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247642,-0.260121,2013.0,0.0,-0.438056
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247651,-0.260116,2014.0,1.0,-0.189088
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247799,-0.260182,2015.0,2.0,-0.295440
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247799,-0.260182,2016.0,3.0,-0.343338
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247799,-0.260182,2017.0,4.0,-0.289197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20977,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.299713,0.799796,0.148532,2016.0,20977.0,0.289439
20978,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.299713,0.799796,0.148532,2017.0,20978.0,0.290574
20979,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.299713,0.799796,0.148532,2018.0,20979.0,0.387619
20980,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.299713,0.799796,0.148532,2019.0,20980.0,-0.081658


In [14]:
#Generate a comprehensive summary for the DataFrame

# Number of missing values per column
missing_count = data_enc.isna().sum()

# Percentage of missing values per column
missing_percentage = (data_enc.isna().mean() * 100).round(2)

# Number of unique values per column
unique_count = data_enc.nunique()

# Number of duplicate rows in the DataFrame
duplicate_count = data_enc.duplicated().sum()


# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage,
    'Unique Count': unique_count
})

print("Comprehensive Data Summary:")

print(summary_df)
print(f"\nNumber of duplicate rows in the dataset: {duplicate_count}")
len(data_enc)

Comprehensive Data Summary:
                    Missing Count  Missing Percentage (%)  Unique Count
who_region_2_Amr                0                     0.0             2
who_region_3_Sear               0                     0.0             2
who_region_4_Eur                0                     0.0             2
who_region_5_Emr                0                     0.0             2
who_region_6_Wpr                0                     0.0             2
...                           ...                     ...           ...
latitude                        0                     0.0          7819
longitude                       0                     0.0          7817
year                            0                     0.0            13
unique_id                       0                     0.0         20982
pm25_concentration              0                     0.0         12335

[142 rows x 3 columns]

Number of duplicate rows in the dataset: 0


20982

In [15]:
data

,who_region,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,type_of_stations,population,latitude,longitude,simplified_station_type,encoded_station_type,encoded_station_type_imputed,final_station_type
0,4_Eur,Spain,A Coruna,2013.0,23.238,11.491,28.841,"Urban, Urban, Suburban",246056.0,43.367900,-8.418571,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
1,4_Eur,Spain,A Coruna,2014.0,27.476,15.878,19.575,"Urban, Urban, Suburban",246056.0,43.368033,-8.418233,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
2,4_Eur,Spain,A Coruna,2015.0,25.515,14.004,22.731,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
3,4_Eur,Spain,A Coruna,2016.0,23.057,13.160,20.204,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
4,4_Eur,Spain,A Coruna,2017.0,26.849,14.114,21.543,"Urban, Urban, Suburban, Suburban",246056.0,43.370375,-8.422900,"Suburban, Urban",4.0,4.0,"Suburban, Urban"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40093,6_Wpr,Republic of Korea,경기도,2017.0,57.335,36.457,0.029,<NA>,NaN,37.337200,126.724100,unknown,NaN,1.8,Rural
40094,6_Wpr,Republic of Korea,경기도,2018.0,50.838,31.586,0.027,<NA>,NaN,37.337200,126.724100,unknown,NaN,1.8,Rural
40095,6_Wpr,Republic of Korea,경기도,2019.0,55.568,31.013,0.028,<NA>,NaN,37.337200,126.724100,unknown,NaN,1.0,Urban
40096,6_Wpr,China,虎英公园北,2018.0,NaN,30.649,NaN,<NA>,NaN,23.012778,113.794444,unknown,NaN,1.4,Urban


In [16]:
data_enc.head()

,who_region_2_Amr,who_region_3_Sear,who_region_4_Eur,who_region_5_Emr,who_region_6_Wpr,who_region_7_NonMS,country_name_Albania,country_name_Algeria,country_name_Argentina,country_name_Australia,...,final_station_type_Suburban,"final_station_type_Suburban, Urban",final_station_type_Traffic,final_station_type_Urban,population,latitude,longitude,year,unique_id,pm25_concentration
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247642,-0.260121,2013.0,0.0,-0.438056
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247651,-0.260116,2014.0,1.0,-0.189088
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247799,-0.260182,2015.0,2.0,-0.295440
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247799,-0.260182,2016.0,3.0,-0.343338
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.199658,0.247799,-0.260182,2017.0,4.0,-0.289197


# Model

In [17]:
X = data_enc.drop(columns=['pm25_concentration' , 'unique_id'])
y = data_enc['pm25_concentration']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Define the Random Forest Regressor with specified parameters
params = {
    'bootstrap': True,
    'ccp_alpha': 0.0,
    'criterion': 'squared_error',
    'max_depth': None,
    'max_features': 1.0,
    'max_leaf_nodes': None,
    'max_samples': None,
    'min_impurity_decrease': 0.0,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0,
    'n_estimators': 100,
    'n_jobs': -1,
    'oob_score': False,
    'random_state': 42,
    'verbose': 0,
    'warm_start': False
}

In [20]:
rf_model = RandomForestRegressor(**params)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [21]:
# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R2): {r2:.4f}")


Mean Absolute Error (MAE): 0.1360
Mean Squared Error (MSE): 0.1044
Root Mean Squared Error (RMSE): 0.3232
R-squared (R2): 0.8891


In [16]:
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)
print(f"Cross-Validation RMSE: {cv_rmse_scores}")
print(f"Average Cross-Validation RMSE: {cv_rmse_scores.mean()}")


Cross-Validation RMSE: [0.50506403 0.44809259 0.39815952 0.37023893 0.39816355]
Average Cross-Validation RMSE: 0.42394372407725467


## Model Functions

In [23]:
def train_and_save_model_rf():
    """
    Train a Random Forest Regressor on the provided dataset and save it.

    The model is trained on the preprocessed data and saved to a file for future use.
    """
    # Specify the model file path
    model_dir = '../airpollutionlevels/models/'
    os.makedirs(model_dir, exist_ok=True)  # Create directory if it doesn't exist
    model_filename = os.path.join(model_dir, 'random_forest_model.pkl')
    df = pd.read_csv('../airpollutionlevels/raw_data/air_pollution_data_encoded_rf.csv')

    # Split the data into features (X) and target (y)
    X = df.drop(columns=['pm25_concentration','unique_id'])
    y = df['pm25_concentration']

    # Define Random Forest Regressor parameters
    params = {
        'bootstrap': True,
        'ccp_alpha': 0.0,
        'criterion': 'squared_error',
        'max_depth': None,
        'max_features': 1.0,
        'max_leaf_nodes': None,
        'max_samples': None,
        'min_impurity_decrease': 0.0,
        'min_samples_leaf': 1,
        'min_samples_split': 2,
        'min_weight_fraction_leaf': 0.0,
        'n_estimators': 100,
        'n_jobs': -1,
        'oob_score': False,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False
    }

    # Initialize the Random Forest Regressor
    rf = RandomForestRegressor(**params)

    # Fit the model
    rf.fit(X, y)

    # Save the model
    joblib.dump(rf, model_filename)
    print(f"Model saved at {model_filename}")

In [24]:
def evaluate_model_rf(test_size=0.2, random_state=123):
    """
    Train a Random Forest model, split the dataset into train and test sets, and evaluate the model's performance.

    Parameters:
    test_size (float): Proportion of the dataset to include in the test split.
    random_state (int): Random seed for reproducibility.

    """
    df = pd.read_csv('../airpollutionlevels/raw_data/air_pollution_data_encoded_rf.csv')
    # Split the data into features (X) and target (y)
    X = df.drop(columns=['pm25_concentration'])
    y = df['pm25_concentration']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model_filename = '../airpollutionlevels/models/random_forest_model.pkl'

    # Load the model
    rf = joblib.load(model_filename)


    # Fit the model on the training data
    rf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rf.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R-squared (R2): {r2:.4f}")


In [30]:
def predict_rf(city, year):
    """
    Load a saved Random Forest model and predict pollution level (PM2.5) for a given city and year.
    If the city does not exist in the dataset, raise an error.

    Parameters:
    city (str): Name of the city.
    year (int): Year for the prediction.

    Returns:
    float: Predicted PM2.5 concentration.
    """
    pm25_scaler_filename = '../airpollutionlevels/models/pm25_scaler.pkl'
    model_filename = '../airpollutionlevels/models/random_forest_model.pkl'
    data = pd.read_csv('../airpollutionlevels/raw_data/data_lib_rf.csv')

    # Verify if the 'city' column exists in the DataFrame
    if 'city' not in data.columns:
        raise ValueError("'city' column not found in the dataset. Please ensure the dataset contains a 'city' column.")

    # Check if the city exists in the dataset
    if city not in data['city'].values:
        raise ValueError(f"No data found for '{city}' in the dataset.")

    # Find the latest row for the provided city in the original dataset
    city_data = data[(data['city'] == city) & (data['year'] == data[data['city'] == city]['year'].max())].copy()

    # Add a unique identifier before encoding
    city_data.loc[:, 'unique_id'] = 0  # Set a temporary unique_id for tracking

    # Encode and scale the entire data
    encoded_data = encode_scale_data_rf(data)

    # Locate the corresponding row in the encoded data using the unique identifier
    encoded_city_data = encoded_data[encoded_data['unique_id'] == 0].copy()

    # Replace the year in the encoded city data with the input year
    encoded_city_data.loc[:, 'year'] = year

    # Drop the unique_id column as it's no longer needed
    encoded_city_data = encoded_city_data.drop(columns=['unique_id'])

    # Load the model
    rf_model = joblib.load(model_filename)

    # Predict pm25_concentration
    prediction = rf_model.predict(encoded_city_data.drop(columns=['pm25_concentration']))[0]

    # Load the pm25 scaler
    pm25_scaler = joblib.load(pm25_scaler_filename)

    # Inverse transform the predicted value
    predicted_pm25 = pm25_scaler.inverse_transform([[prediction]])[0][0]

    print(f"Predicted PM2.5 concentration for {city} in {year}: {predicted_pm25}")
    return predicted_pm25

In [26]:
train_and_save_model_rf

<function __main__.train_and_save_model_rf()>

In [27]:
evaluate_model_rf()

Mean Absolute Error (MAE): 0.1350
Mean Squared Error (MSE): 0.1018
Root Mean Squared Error (RMSE): 0.3191
R-squared (R2): 0.8872


In [28]:
predict_rf('Berlin', 2018)

Predicted PM2.5 concentration for Berlin in 2018: 13.716160000000006


13.716160000000006

In [29]:
data = pd.read_csv('../airpollutionlevels/raw_data/data_lib_rf.csv')

temp = data[(data['city'] == 'Berlin') & (data['year'] == 2018)]
temp

,who_region,country_name,city,year,pm10_concentration,pm25_concentration,no2_concentration,type_of_stations,population,latitude,longitude,simplified_station_type,encoded_station_type,encoded_station_type_imputed,final_station_type
3616,4_Eur,Germany,Berlin,2018.0,23.739,15.514,38.5,"Urban, Urban, Rural, Rural, Urban, Suburban, R...",11313.0,52.503477,13.392864,"Rural, Suburban, Urban",6.0,6.0,"Rural, Suburban, Urban"
